# TO DO

3. Check other cities by downloading their data. Is there an online source?

AWS NAIP:
https://aws.amazon.com/public-datasets/naip/

See naming specifications for NAIP here: 
https://lta.cr.usgs.gov/naip_full_res.html

They are based on the 7.5 by 7.5 Minute USGS quadrangle: 
https://topomaps.usgs.gov/drg/drg_name.html#hdr%209

In [1]:
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
def get_quad_id(lat, lon):
    
    ## Assume 8x8 grid. 
    grid_size = 8
    grid_step = 1 / grid_size 
    
    ## Find index in 1-degree block (8x8 grid)
    ## NOTE: Quad are 1-indexed
    lon_degrees = lon - int(lon)
    lat_degrees = lat - int(lat)
    col_ndx = (grid_size - 1) + int(lon_degrees / grid_step)
    row_ndx = (grid_size - 1) - int(lat_degrees / grid_step)
    ndx = 1 + np.ravel_multi_index((row_ndx , col_ndx), (grid_size, grid_size))
    
    ## Find quadrant (nw, ne, sw, se)
    lon_upper = int(lon) - grid_step * (grid_size - col_ndx - 1)
    lon_lower = int(lon) - grid_step * (grid_size - col_ndx)
    lat_upper = int(lat) + grid_step * (grid_size - row_ndx)
    lat_lower = int(lat) + grid_step * (grid_size - row_ndx - 1)
    lon_mid = (lon_upper + lon_lower) / 2
    lat_mid = (lat_upper + lat_lower) / 2
    if lat > lat_mid:
        quadrant = 'n'
    else:
        quadrant = 's'
        
    if lon > lon_mid:
        quadrant = quadrant + 'e'
    else:
        quadrant = quadrant + 'w'
    
    ## Format output string
    lat_str = str(int(lat))
    if abs(lon) < 100:
        lon_str = '0' + str(abs(int(lon)))
    else:
        lon_str = str(abs(int(lon)))
    
    return lat_str + lon_str + str(ndx) + '_' + quadrant

In [3]:
places = [
    'Arizona State University AZ',
    'Northern Arizona University AZ',
    'University of Arizona AZ',
    'Georgia Tech GA',
    'New York University NY',
    'Empire State Building NY',
    'Brooklyn Botanic Garden NY',
    'Calvary Cemetery NY'
]

In [4]:
for place in places:
    geolocator = Nominatim()
    loc = geolocator.geocode(place)
    lat, lon = loc.latitude, loc.longitude
    quad_id = get_quad_id(lat, lon)
    
    print(place)
    print(lat, lon)
    print(quad_id)
    print()

Arizona State University AZ
33.42165145 -111.932582859275
3311133_se

Northern Arizona University AZ
35.1851087 -111.655837039036
3511151_se

University of Arizona AZ
33.45205165 -112.063678942176
3311240_nw

Georgia Tech GA
33.776033 -84.3988408600158
3308413_se

New York University NY
40.7292641 -73.9962682727708
4007317_nw

Empire State Building NY
40.7484284 -73.9856546198733
4007317_nw

Brooklyn Botanic Garden NY
40.66760805 -73.9636076711714
4007317_sw

Calvary Cemetery NY
40.73296815 -73.9308954670053
4007317_ne



Download the NAIP manifest from AWS and grep through the listed files for the quadrangle ID.

```shell
aws s3api get-object --bucket aws-naip --key manifest.txt --request-payer requester manifest.txt
grep "4007317" manifest.txt
```

returns a list with all files corresponding to that ID

```
ny/2013/1m/fgdc/40073/m_4007317_ne_18_1_20130622.txt
ny/2013/1m/fgdc/40073/m_4007317_nw_18_1_20130622.txt
ny/2013/1m/fgdc/40073/m_4007317_se_18_1_20130622.txt
ny/2013/1m/fgdc/40073/m_4007317_sw_18_1_20130622.txt
ny/2013/1m/rgb/40073/m_4007317_ne_18_1_20130622.tif
ny/2013/1m/rgb/40073/m_4007317_nw_18_1_20130622.tif
ny/2013/1m/rgb/40073/m_4007317_se_18_1_20130622.tif
ny/2013/1m/rgb/40073/m_4007317_sw_18_1_20130622.tif
ny/2013/1m/rgbir/40073/m_4007317_ne_18_1_20130622.tif
ny/2013/1m/rgbir/40073/m_4007317_nw_18_1_20130622.tif
ny/2013/1m/rgbir/40073/m_4007317_se_18_1_20130622.tif
ny/2013/1m/rgbir/40073/m_4007317_sw_18_1_20130622.tif
ny/2015/.5m/fgdc/40073/m_4007317_ne_18_h_20150522.txt
ny/2015/.5m/fgdc/40073/m_4007317_nw_18_h_20150522.txt
ny/2015/.5m/fgdc/40073/m_4007317_se_18_h_20150522.txt
ny/2015/.5m/fgdc/40073/m_4007317_sw_18_h_20150522.txt
ny/2015/.5m/rgb/40073/m_4007317_ne_18_h_20150522.tif
ny/2015/.5m/rgb/40073/m_4007317_nw_18_h_20150522.tif
ny/2015/.5m/rgb/40073/m_4007317_se_18_h_20150522.tif
ny/2015/.5m/rgb/40073/m_4007317_sw_18_h_20150522.tif
ny/2015/.5m/rgbir/40073/m_4007317_ne_18_h_20150522.tif
ny/2015/.5m/rgbir/40073/m_4007317_nw_18_h_20150522.tif
ny/2015/.5m/rgbir/40073/m_4007317_se_18_h_20150522.tif
ny/2015/.5m/rgbir/40073/m_4007317_sw_18_h_20150522.tif
```

Files can then be downloaded

```shell
aws s3api get-object --bucket aws-naip --key ny/2013/1m/rgb/40073/m_4007317_ne_18_1_20130622.tif --request-payer requester m_4007317_ne_18_1_20130622.tif

aws s3api get-object --bucket aws-naip --key ny/2013/1m/rgb/40073/m_4007317_sw_18_1_20130622.tif --request-payer requester m_4007317_sw_18_1_20130622.tif
```

In [18]:
lat = 40.7484284
lon = -73.9856546198733

lon_degrees = lon - int(lon)
lat_degrees = lat - int(lat)
col_ndx = 7 + int(lon_degrees / 0.125)
row_ndx = 7 - int(lat_degrees / 0.125)

lon_upper = int(lon) - .125*(7-col_ndx)
lon_lower = int(lon) - .125*(7-col_ndx+1)
lat_upper = int(lat) + .125*(7-row_ndx+1)
lat_lower = int(lat) + .125*(7-row_ndx)

lon_mid = (lon_upper + lon_lower) / 2
lat_mid = (lat_upper + lat_lower) / 2

if lat > lat_mid:
    quadrant = 'n'
else:
    quadrant = 's'
    
if lon > lon_mid:
    quadrant = quadrant + 'e'
else:
    quadrant = quadrant + 'w'

print('Lon: {} {}'.format(lon_lower, lon_upper))
print('Lat: {} {}'.format(lat_lower, lat_upper))
print(quadrant)


Lon: -74.0 -73.875
Lat: 40.625 40.75
nw
